In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
from Utils.Data_processing import *

df = pd.read_csv(r'Data\train.csv')
print(df.shape)
df.head()

Segmentation Models: using `keras` framework.
(115488, 3)


,id,class,segmentation
0,case123_day20_slice_0001,large_bowel,NaN
1,case123_day20_slice_0001,small_bowel,NaN
2,case123_day20_slice_0001,stomach,NaN
3,case123_day20_slice_0002,large_bowel,NaN
4,case123_day20_slice_0002,small_bowel,NaN


In [2]:
df = pd.read_csv(r'Data\train.csv')
print(df.shape)
df.head()

(115488, 3)


,id,class,segmentation
0,case123_day20_slice_0001,large_bowel,NaN
1,case123_day20_slice_0001,small_bowel,NaN
2,case123_day20_slice_0001,stomach,NaN
3,case123_day20_slice_0002,large_bowel,NaN
4,case123_day20_slice_0002,small_bowel,NaN


In [4]:
df.rename(columns = {'class':'class_name'}, inplace = True)
#--------------------------------------------------------------------------
df["case"] = df["id"].apply(lambda x: int(x.split("_")[0].replace("case", "")))
df["day"] = df["id"].apply(lambda x: int(x.split("_")[1].replace("day", "")))
df["slice"] = df["id"].apply(lambda x: x.split("_")[3])
df.head()

,id,class_name,segmentation,case,day,slice
0,case123_day20_slice_0001,large_bowel,NaN,123,20,0001
1,case123_day20_slice_0001,small_bowel,NaN,123,20,0001
2,case123_day20_slice_0001,stomach,NaN,123,20,0001
3,case123_day20_slice_0002,large_bowel,NaN,123,20,0002
4,case123_day20_slice_0002,small_bowel,NaN,123,20,0002


In [5]:
TRAIN_DIR="Data/train"
# get all the image path with glob
all_train_images = glob(os.path.join(TRAIN_DIR, "**", "*.png"), recursive=True)
x = all_train_images[0].rsplit("/", 4)[0] ## ../input/uw-madison-gi-tract-image-segmentation/train

path_partial_list = []
for i in range(0, df.shape[0]):
    path_partial_list.append(os.path.join(x,
                          "case"+str(df["case"].values[i]),
                          "case"+str(df["case"].values[i])+"_"+ "day"+str(df["day"].values[i]),
                          "scans",
                          "slice_"+str(df["slice"].values[i])))
df["path_partial"] = path_partial_list
df['path_partial'] = df['path_partial'].apply(lambda x: x.replace("Data","Data/train"))
df.head()

,id,class_name,segmentation,case,day,slice,path_partial
0,case123_day20_slice_0001,large_bowel,NaN,123,20,0001,Data/train\case123\case123_day20\scans\slice_0001
1,case123_day20_slice_0001,small_bowel,NaN,123,20,0001,Data/train\case123\case123_day20\scans\slice_0001
2,case123_day20_slice_0001,stomach,NaN,123,20,0001,Data/train\case123\case123_day20\scans\slice_0001
3,case123_day20_slice_0002,large_bowel,NaN,123,20,0002,Data/train\case123\case123_day20\scans\slice_0002
4,case123_day20_slice_0002,small_bowel,NaN,123,20,0002,Data/train\case123\case123_day20\scans\slice_0002


In [6]:
path_partial_list = []
for i in range(0, len(all_train_images)):
    path_partial_list.append(str(all_train_images[i].rsplit("_",4)[0]))
    
tmp_df = pd.DataFrame()
tmp_df['path_partial'] = path_partial_list
tmp_df['path'] = all_train_images


In [7]:
df = df.merge(tmp_df, on="path_partial").drop(columns=["path_partial"])
# -4 as we don't need the '.png'
df["width"] = df["path"].apply(lambda x: int(x[:-3].rsplit("_",4)[1]))
df["height"] = df["path"].apply(lambda x: int(x[:-3].rsplit("_",4)[2]))
#--------------------------------------------------------------------------
del x,path_partial_list,tmp_df
#--------------------------------------------------------------------------
print(df.shape)
df.head(5)

(115488, 9)


,id,class_name,segmentation,case,day,slice,path,width,height
0,case123_day20_slice_0001,large_bowel,NaN,123,20,0001,Data/train\case123\case123_day20\scans\slice_0...,266,266
1,case123_day20_slice_0001,small_bowel,NaN,123,20,0001,Data/train\case123\case123_day20\scans\slice_0...,266,266
2,case123_day20_slice_0001,stomach,NaN,123,20,0001,Data/train\case123\case123_day20\scans\slice_0...,266,266
3,case123_day20_slice_0002,large_bowel,NaN,123,20,0002,Data/train\case123\case123_day20\scans\slice_0...,266,266
4,case123_day20_slice_0002,small_bowel,NaN,123,20,0002,Data/train\case123\case123_day20\scans\slice_0...,266,266


In [8]:
# RESTRUCTURE  DATAFRAME
df_train = pd.DataFrame({'id':df['id'][::3]})

df_train['large_bowel'] = df['segmentation'][::3].values
df_train['small_bowel'] = df['segmentation'][1::3].values
df_train['stomach'] = df['segmentation'][2::3].values

df_train['path'] = df['path'][::3].values
df_train['case'] = df['case'][::3].values
df_train['day'] = df['day'][::3].values
df_train['slice'] = df['slice'][::3].values
df_train['width'] = df['width'][::3].values
df_train['height'] = df['height'][::3].values


df_train.reset_index(inplace=True,drop=True)
df_train.fillna('',inplace=True); 
# the count columns store the number of mask the exist in image
df_train['count'] = np.sum(df_train.iloc[:,1:4]!='',axis=1).values
print(df_train.shape)
df_train.sample(5)

(38496, 11)


,id,large_bowel,small_bowel,stomach,path,case,day,slice,width,height,count
1301,case129_day20_slice_0006,,,,Data/train\case129\case129_day20\scans\slice_0...,129,20,0006,360,310,0
6432,case142_day0_slice_0017,,,,Data/train\case142\case142_day0\scans\slice_00...,142,0,0017,266,266,0
36353,case80_day0_slice_0018,,,,Data/train\case80\case80_day0\scans\slice_0018...,80,0,0018,266,266,0
23660,case143_day0_slice_0141,,,,Data/train\case143\case143_day0\scans\slice_01...,143,0,0141,266,266,0
5110,case42_day17_slice_0071,,,,Data/train\case42\case42_day17\scans\slice_007...,42,17,0071,266,266,0


In [9]:
df_train.iloc[0]['path']

'Data/train\\case123\\case123_day20\\scans\\slice_0001_266_266_1.50_1.50.png'

In [ ]:
plt.figure(figsize=(20,20))

In [11]:
BATCH_SIZE=1

In [13]:
Masks = list(df_train[df_train['large_bowel']!=''].sample(BATCH_SIZE).index)
Masks

[24522]